In [14]:
import pandas as pd
import yaml

customer_statistics = '/content/sample_data/customer_statistics.csv'
orders = '/content/sample_data/orders.csv'
customer_demographics = '/content/sample_data/customer_demographics.yaml'


with open(customer_demographics, 'r') as file:
    data = yaml.safe_load(file)

df_customer_demographics = pd.DataFrame(data).transpose()


In [15]:
df_customer_demographics

,address,city,credit_card_expires,credit_card_number,credit_card_provider,credit_card_security_code,customer_id,email,name,phone_number,state,zip_code
0,621 Chavez Ramp Suite 938,West Deniseport,06/31,2269172571677458,Diners Club / Carte Blanche,338,a8c966a0-b670-4044-826c-0e6bfe708259,brett00@example.com,Ronald Rodgers,None,New York,59237
1,592 Mccoy River,Port Dannymouth,11/24,676320507830,JCB 16 digit,246,56b850e0-9320-4aa0-8cd4-d1199903c90e,smithjennifer@example.net,Mary Mitchell,None,Montana,17401
2,8923 Aaron Turnpike,Port Rhondaton,11/31,4607779976715144588,JCB 16 digit,869,5ec9f613-adb1-4e69-8a1a-9530cb777886,twhite@example.net,Nathan Wells,+1-893-568-2182x593,North Dakota,07166
3,496 Margaret Shoals,Smithborough,04/33,4234465830748237,Mastercard,497,ca182662-b281-433a-a82b-8e061f7bf2cd,nguyenjennifer@example.com,Kelly Ramos,820-824-2003,Iowa,42755
4,0807 Logan Fords Suite 355,Contrerasside,10/25,4985681645741,Discover,927,1e7c3a60-4788-4711-809f-b6f8bfbdb18a,michaelabarr@example.net,Rebekah Fitzgerald,+1-269-384-1623x34240,Connecticut,64282
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,419 Lopez Crossing,North Nicolechester,01/30,639083591672,VISA 13 digit,079,0707f817-5018-4060-b01c-2c5b53374a2e,andrewrogers@example.net,Tracey Edwards,001-555-466-6738x7224,Alabama,15476
9996,38594 Franklin Pine,Port Joshua,01/31,4197551271949528,VISA 13 digit,237,abc92c1c-788c-4e61-a40d-c6754c99bdc7,sduncan@example.com,Ashley Peterson,(434)761-1066,New Jersey,58035
9997,535 Kimberly Station Apt. 546,Garzaville,07/24,36597049683777,VISA 16 digit,287,418cc301-e111-4080-b8f7-8d9f1d9dc304,rhatfield@example.net,Sandra Baldwin MD,867.829.9891x64114,Connecticut,16016
9998,35316 Becky Terrace,Port Donald,04/24,6595168672629675,JCB 16 digit,113,7caf0251-8040-4c1e-8190-46133fde16ab,andreawilliams@example.org,Karen Green,None,Nevada,97426


In [16]:
df_customer_statistics = pd.read_csv(customer_statistics)

In [17]:
df_customer_statistics

,customer_id,total_orders,total_items,total_spent
0,8037,5,20,300.39
1,4290,2,3,332.66
2,5367,3,7,643.27
3,4755,10,12,649.12
4,2651,10,9,794.28
...,...,...,...,...
9995,585,10,14,394.88
9996,4604,2,1,858.88
9997,5168,7,8,137.52
9998,3722,5,1,105.75


In [18]:
#clean
clean_df_customer_statistics = df_customer_statistics.groupby('customer_id').agg({
    'total_orders': 'sum',
    'total_items': 'sum',
    'total_spent': 'sum'}).reset_index()

clean_df_customer_statistics

,customer_id,total_orders,total_items,total_spent
0,0,12,29,2299.91
1,3,7,17,392.78
2,4,17,25,1789.48
3,5,4,21,803.61
4,6,9,39,1935.00
...,...,...,...,...
6278,9993,7,9,405.65
6279,9994,2,10,338.05
6280,9995,1,9,864.16
6281,9996,6,13,649.99


In [19]:
df_orders = pd.read_csv(orders)

df_orders

,order_id,customer_id,items,aperitifs,appetizers,entrees,desserts,total
0,f1d6b4c4-0f9f-4ba6-8d5d-f7e4ef7cd9a0,2652,7,Aperol Spritz,Pate,Pork,Tiramisu,751.87
1,7b8a732e-85a0-4fa2-b282-a2d232ee182e,1756,3,Sherry,Pate,Pasta,Cheesecake,156.43
2,1eab436d-2acb-4bc8-b849-fbce859ffd8a,2989,5,Bellini,Pate,Lobster,Chocolate Cake,364.13
3,d6313392-8901-42f6-bf8a-4805fe816a46,9903,3,Sherry,Salad,Pizza,Cheesecake,805.62
4,f73f8c45-28c2-473f-a3ba-e65ed5890ac6,8627,2,Port,Pate,Lobster,Pie,638.43
...,...,...,...,...,...,...,...,...
9995,bc3dd6ac-88e3-4978-8364-c047fcf44db2,5355,2,Port,Oysters,Vegetarian,Cookies,466.83
9996,7115ef6e-f8ab-4847-856a-3463d98618d5,4452,7,Port,Escargot,Lobster,Pie,311.93
9997,b03408a5-e879-420d-b3dd-2b0efa1bee17,2982,6,Port,Foie Gras,Vegetarian,Ice Cream,315.54
9998,8d611f13-8886-48a1-9805-30beeab154d9,5106,4,Vermouth,Crudites,Pasta,Sorbet,131.74


In [24]:
#merge first round
merged_final = pd.merge(clean_df_customer_statistics, df_customer_demographics, left_on='customer_id', right_index=True, how='inner')
merged_final


,customer_id,customer_id_x,total_orders,total_items,total_spent,address,city,credit_card_expires,credit_card_number,credit_card_provider,credit_card_security_code,customer_id_y,email,name,phone_number,state,zip_code
0,0,0,12,29,2299.91,621 Chavez Ramp Suite 938,West Deniseport,06/31,2269172571677458,Diners Club / Carte Blanche,338,a8c966a0-b670-4044-826c-0e6bfe708259,brett00@example.com,Ronald Rodgers,None,New York,59237
1,3,3,7,17,392.78,496 Margaret Shoals,Smithborough,04/33,4234465830748237,Mastercard,497,ca182662-b281-433a-a82b-8e061f7bf2cd,nguyenjennifer@example.com,Kelly Ramos,820-824-2003,Iowa,42755
2,4,4,17,25,1789.48,0807 Logan Fords Suite 355,Contrerasside,10/25,4985681645741,Discover,927,1e7c3a60-4788-4711-809f-b6f8bfbdb18a,michaelabarr@example.net,Rebekah Fitzgerald,+1-269-384-1623x34240,Connecticut,64282
3,5,5,4,21,803.61,9290 Krista Cliff Suite 506,West Kaylastad,09/27,371450682302318,Maestro,577,a69d43cc-6c80-4424-b2bc-1cd0c59757f8,josephaustin@example.com,Diana Silva,895.892.9318,Massachusetts,39877
4,6,6,9,39,1935.00,349 Martinez Turnpike,Port Mikaylashire,11/24,2364648842479646,Discover,155,6c84a42c-895d-4c26-8db7-99e0e01ddb6f,quinnrachael@example.net,Paul Reed,None,New Hampshire,63162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6278,9993,9993,7,9,405.65,45292 Amy Stream,Martinezland,11/30,4011981090951526,JCB 16 digit,849,12f10986-8ece-46d5-b853-1fcb7cbc270d,None,Gabrielle Carter,001-816-334-9707,Vermont,65616
6279,9994,9994,2,10,338.05,73248 Angela Plain Apt. 444,East Larryfort,04/25,502023332864,VISA 16 digit,002,352141f4-c26c-4973-9afd-2d1540f28bee,christianmoody@example.net,Lisa Rogers,273-633-4909x2896,Oklahoma,84716
6280,9995,9995,1,9,864.16,419 Lopez Crossing,North Nicolechester,01/30,639083591672,VISA 13 digit,079,0707f817-5018-4060-b01c-2c5b53374a2e,andrewrogers@example.net,Tracey Edwards,001-555-466-6738x7224,Alabama,15476
6281,9996,9996,6,13,649.99,38594 Franklin Pine,Port Joshua,01/31,4197551271949528,VISA 13 digit,237,abc92c1c-788c-4e61-a40d-c6754c99bdc7,sduncan@example.com,Ashley Peterson,(434)761-1066,New Jersey,58035


In [29]:
#merge second round
merged_final_final = pd.merge(merged_final, df_orders, on='customer_id', how='right')
merged_final_final

,customer_id,customer_id_x,total_orders,total_items,total_spent,address,city,credit_card_expires,credit_card_number,credit_card_provider,...,phone_number,state,zip_code,order_id,items,aperitifs,appetizers,entrees,desserts,total
0,2652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,f1d6b4c4-0f9f-4ba6-8d5d-f7e4ef7cd9a0,7,Aperol Spritz,Pate,Pork,Tiramisu,751.87
1,1756,1756.0,2.0,15.0,727.48,374 Jamie Ways Apt. 220,West Carlos,09/26,3550419833027998,VISA 19 digit,...,None,Wisconsin,01709,7b8a732e-85a0-4fa2-b282-a2d232ee182e,3,Sherry,Pate,Pasta,Cheesecake,156.43
2,2989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1eab436d-2acb-4bc8-b849-fbce859ffd8a,5,Bellini,Pate,Lobster,Chocolate Cake,364.13
3,9903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,d6313392-8901-42f6-bf8a-4805fe816a46,3,Sherry,Salad,Pizza,Cheesecake,805.62
4,8627,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,f73f8c45-28c2-473f-a3ba-e65ed5890ac6,2,Port,Pate,Lobster,Pie,638.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,5355,5355.0,18.0,29.0,1355.88,08295 Nathan Springs Apt. 862,Kathleenshire,04/24,3539227365299158,JCB 16 digit,...,001-385-936-5914x26678,Oregon,75946,bc3dd6ac-88e3-4978-8364-c047fcf44db2,2,Port,Oysters,Vegetarian,Cookies,466.83
9996,4452,4452.0,1.0,19.0,545.92,None,None,11/32,4264078472869536587,Diners Club / Carte Blanche,...,734-568-5349,None,None,7115ef6e-f8ab-4847-856a-3463d98618d5,7,Port,Escargot,Lobster,Pie,311.93
9997,2982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,b03408a5-e879-420d-b3dd-2b0efa1bee17,6,Port,Foie Gras,Vegetarian,Ice Cream,315.54
9998,5106,5106.0,16.0,32.0,879.38,099 Harris Key Suite 746,Coopertown,03/31,36368094216156,VISA 16 digit,...,429.882.8378x5414,Iowa,46867,8d611f13-8886-48a1-9805-30beeab154d9,4,Vermouth,Crudites,Pasta,Sorbet,131.74
